# 학습용 데이터 전체기간 수집
- 내가 맡은 연도 2012~2014
- 월은 sample3와는 달리 전체 월
- 지역도 모든 지역 다
- 위,경도는 카카오 API로
- 기상데이터는 해당 산불이 일어난 지역의 기상으로

In [1]:
# 셋팅
import pandas as pd
import numpy as np
from google.colab import files
import requests
import re
import pymongo
from scipy.spatial import distance
import datetime

## 실효습도에서 지역을 전체지역으로 선택하고 크롤링하면 asos와 같은 지점갯수가 나오는지 확인

In [2]:
# df = pd.read_csv('/content/STCS_실효습도_20220706134232.csv', encoding = 'cp949')
# df

In [3]:
# df['지점번호'].unique()

In [4]:
# len(df['지점번호'].unique())

In [5]:
# df[df['지점번호'] == 175]

위경도 데이터 지점 갯수 확인

In [114]:
latitude_df = pd.read_csv('/content/drive/MyDrive/DE  DS 합반 (산불위험 프젝)/데이터셋/mongo_db,관측지점,위경도(0701.ver).csv')
latitude_df

,_id,지점번호,지점명,위도,경도
0,62be6fb4a82199005252108d,90,속초,38.25085,128.56473
1,62be6fb4a82199005252108e,93,북춘천,37.94738,127.75443
2,62be6fb4a82199005252108f,95,철원,38.14787,127.30420
3,62be6fb4a821990052521090,98,동두천,37.90188,127.06070
4,62be6fb4a821990052521091,99,파주,37.88589,126.76648
...,...,...,...,...,...
90,62be6fb4a8219900525210e7,285,합천,35.56505,128.16994
91,62be6fb4a8219900525210e8,288,밀양,35.49147,128.74413
92,62be6fb4a8219900525210e9,289,산청,35.41300,127.87910
93,62be6fb4a8219900525210ea,294,거제,34.88818,128.60458


In [7]:
latitude_df['지점번호'].unique()

array([ 90,  93,  95,  98,  99, 100, 101, 102, 104, 105, 106, 108, 112,
       114, 115, 119, 121, 127, 129, 130, 131, 133, 135, 136, 137, 138,
       140, 143, 146, 152, 155, 156, 159, 162, 165, 168, 169, 170, 172,
       174, 177, 184, 185, 188, 189, 192, 201, 202, 203, 211, 212, 216,
       217, 221, 226, 232, 235, 236, 238, 239, 243, 244, 245, 247, 248,
       251, 252, 253, 254, 255, 257, 258, 259, 260, 261, 262, 263, 264,
       266, 268, 271, 272, 273, 276, 277, 278, 279, 281, 283, 284, 285,
       288, 289, 294, 295])

In [8]:
len(latitude_df['지점번호'].unique())

95

설립일 확인

In [9]:
std_df = pd.read_csv('/content/drive/MyDrive/DE  DS 합반 (산불위험 프젝)/데이터셋/ASOS_관측지점_시작날짜.csv')
std_df

,지점번호,지점명,시작시각
0,90,속초,1968.01.01
1,93,북춘천,2016.10.01
2,95,철원,1988.01.01
3,98,동두천,1998.02.01
4,99,파주,2001.12.07
...,...,...,...
90,285,합천,1973.01.01
91,288,밀양,1973.01.01
92,289,산청,1972.03.30
93,294,거제,1972.01.24


In [10]:
std_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지점번호    95 non-null     int64 
 1   지점명     95 non-null     object
 2   시작시각    95 non-null     object
dtypes: int64(1), object(2)
memory usage: 2.4+ KB


In [11]:
lst = []

for a in std_df['시작시각']:

  a = a.replace('.','')
  lst.append(a)

std_df['시작시각'] = lst

In [12]:
std_df[(std_df['시작시각'] >= '20120101') & (std_df['시작시각'] < '20140620')]

,지점번호,지점명,시작시각
79,268,진도군,20140509


In [13]:
# 2014년 이후부터 관측을 시작한 지점 확인
std_df[std_df['시작시각'] > '20140619']

,지점번호,지점명,시작시각
1,93,북춘천,20161001
40,177,홍성,20151103
59,239,세종,20190531


In [14]:
# 2012.01~2014.06 의 기상데이터 및 실효습도를 뽑아낼수없는 관측소 지우기
latitude_df = latitude_df.drop([1,40,59,79], axis=0)
latitude_df = latitude_df.drop('_id', axis=1)  # 지우는 김에 필요없는 컬럼도 삭제
latitude_df

,지점번호,지점명,위도,경도
0,90,속초,38.25085,128.56473
2,95,철원,38.14787,127.30420
3,98,동두천,37.90188,127.06070
4,99,파주,37.88589,126.76648
5,100,대관령,37.67713,128.71834
...,...,...,...,...
90,285,합천,35.56505,128.16994
91,288,밀양,35.49147,128.74413
92,289,산청,35.41300,127.87910
93,294,거제,34.88818,128.60458


In [15]:
# 지점번호 타입캐스팅
latitude_df['지점번호'] = latitude_df['지점번호'].astype(str)

## 기상 데이터 추출 (API 활용)

In [16]:
# API_KEY = "CJg6YBp/KNrl7YueJ48EwyvDpgka/lL0pUeuRvW84Aw+bDjTm0W1HVZSbhxk6mE4kU+jgLex72Up7mofH9hMUQ=="
# URL = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'

# params = {
#     'serviceKey' : API_KEY,
#     'pageNo' : '1', 
#     'numOfRows' : '900', 
#     'dataType' : 'json',
#     'dataCd' : 'ASOS', 
#     'dateCd' : 'HR',
#     'startDt' : '20120101', 
#     'startHh' : '00', 
#     'endDt' : '20140619', 
#     'endHh' : '23', 
#     'stnIds' : '264'
# }

# response = requests.get(URL, params=params)
# datas = response.json()
# datas

In [17]:
# datas['response']['body']['items']['item']

In [18]:
def climate(local,start,start_time,end,end_time):
  API_KEY = "CJg6YBp/KNrl7YueJ48EwyvDpgka/lL0pUeuRvW84Aw+bDjTm0W1HVZSbhxk6mE4kU+jgLex72Up7mofH9hMUQ=="
  URL = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'

  params = {
      'serviceKey' : API_KEY,
      'pageNo' : '1', 
      'numOfRows' : '900', 
      'dataType' : 'json',
      'dataCd' : 'ASOS', 
      'dateCd' : 'HR',
      'startDt' : start, 
      'startHh' : start_time, 
      'endDt' : end, 
      'endHh' : end_time, 
      'stnIds' : local
  }

  response = requests.get(URL, params=params)
  datas = response.json()

  full_data = []

  for a in datas['response']['body']['items']['item']:
    stnnm = a['stnNm']  # 지점명
    tm = a['tm']  # 일시
    ws = a['ws']  # 풍속
    wd = a['wd']  # 풍향
    hm = a['hm']  # 습도
    ta = a['ta']  # 기온
    rn = a['rn']  # 강수량


    data = [tm, stnnm, local, ta, rn, ws, wd, hm]

    full_data.append(data)

  return full_data

# 자동화
climate_datas = []

def plus_list(local,start,start_time,end,end_time):
  global climate_datas 
  climate_datas = climate_datas + climate(local,start,start_time,end,end_time)

  return climate_datas

In [19]:
# # 모든지역 추출 - 지점별 위경도 데이터에서 지점번호만 추출후 for문
# # 한번에 2012부터 2014까지 추출 안됨. 갯수제한때문에
# # 한달 단위로 끊어서 추출
# for a in list(latitude_df['지점번호'].unique()):
#   try:
#     # 2012년
#     plus_list(a,'20120101','00','20120131','23')
#     plus_list(a,'20120201','00','20120229','23')
#     plus_list(a,'20120301','00','20120331','23')
#     plus_list(a,'20120401','00','20120430','23')
#     plus_list(a,'20120501','00','20120531','23')
#     plus_list(a,'20120601','00','20120630','23')
#     plus_list(a,'20120701','00','20120731','23')
#     plus_list(a,'20120801','00','20120831','23')
#     plus_list(a,'20120901','00','20120930','23')
#     plus_list(a,'20121001','00','20121031','23')
#     plus_list(a,'20121101','00','20121130','23')
#     plus_list(a,'20121201','00','20121231','23')

#     # 2013년
#     plus_list(a,'20130101','00','20130131','23')
#     plus_list(a,'20130201','00','20130228','23')
#     plus_list(a,'20130301','00','20130331','23')
#     plus_list(a,'20130401','00','20130430','23')
#     plus_list(a,'20130501','00','20130531','23')
#     plus_list(a,'20130601','00','20130630','23')
#     plus_list(a,'20130701','00','20130731','23')
#     plus_list(a,'20130801','00','20130831','23')
#     plus_list(a,'20130901','00','20130930','23')
#     plus_list(a,'20131001','00','20131031','23')
#     plus_list(a,'20131101','00','20131130','23')
#     plus_list(a,'20131201','00','20131231','23')

#     # 2014년
#     plus_list(a,'20140101','00','20140131','23')
#     plus_list(a,'20140201','00','20140228','23')
#     plus_list(a,'20140301','00','20140331','23')
#     plus_list(a,'20140401','00','20140430','23')
#     plus_list(a,'20140501','00','20140531','23')
#     plus_list(a,'20140601','00','20140619','23')
#   except:
#     print('error!')
#     print(a)

# climate_df = pd.DataFrame(climate_datas)

# # 컬럼명
# climate_df.columns = ['일시', '지점명', '지점번호', '기온', '강수량', '풍속', '풍향', '습도']
# climate_df

2시간 30분 걸림
- 에러뜬 지점번호
  - 95
  - 217
  - 273
  - 279
  - 283
  - 294

In [20]:
# # 중복으로 들어간 데이터 없는지 확인
# climate_df[(climate_df['지점명'] == '속초') & (climate_df['일시'] == '2012-01-01 00:00')]

In [21]:
# climate_df.loc[[804434,804435],:]

In [22]:
# # 중복 데이터 들어감. 인덱스 804434 까지 분리 후 저장
# error_df = climate_df.loc[:804434,:]
# climate_df = climate_df.loc[804435:,:].reset_index(drop=True)
# climate_df.to_csv('기상데이터_1차_초안.csv', index=False)
# files.download('기상데이터_1차_초안.csv')

In [23]:
# # 중복데이터로 에러 원인 찾기
# error_df.to_csv('기상데이터_중복데이터.csv', index=False)
# files.download('기상데이터_중복데이터.csv')

### 오류났던 데이터들 확인
- 몇월에 오류났는지 확인

In [24]:
# # 오류났던 데이터 색인
# a = climate_df.value_counts('지점명')
# b = pd.DataFrame(a, columns=['갯수'])
# b

In [25]:
# b[b['갯수'] != 21624]
# b[b['갯수'] < 20000]

대부분의 경우 지역별로 21624개의 데이터가 모였고 몇개 정도 데이터가 빠지더라도 대부분 2만개는 넘겼음. 에러뜬 
- 95(철원), 217(정선군), 273(문경), 279(구미), 283(경주시), 294(거제)
  
  이 6개는 2만개보다 적은 숫자로 나옴.


해결책  
1. 6개에 해당하는 지역을 지우고 그 지역들만 다시 데이터를 뽑아온다.


In [26]:
# # 95(철원) -> 2012-01-31 23:00 까지 밖에 없음. 
# # 217(정선군) -> 2013-09-30 23:00 까지 존재 15323
# # 273(문경) -> 2012-05-31 23:00 까지 존재 3648
# # 279(구미) -> 	2014-01-31 23:00 까지 존재 18288
# # 283(경주) -> 2012-02-29 23:00 까지 존재 18288
# # 294(거제) -> 	2014-02-28 23:00 까지 존재 18954

# climate_df[climate_df['지점번호'] == '294']

In [27]:
# climate_df[(climate_df['지점번호'] == '217') & (climate_df['일시'] == '2012-01-24 22:00')]

In [28]:
# climate_df[climate_df['일시'] == '2012-01-01 00:00']

추가 크롤링 6개만

In [29]:
# # 95(철원), 217(정선군), 273(문경), 279(구미), 283(경주시), 294(거제)

# lst = ['95', '217', '273', '279', '283', '294']

# for a in lst:
#   try:
#     # 2012년
#     plus_list(a,'20120101','00','20120131','23')
#     plus_list(a,'20120201','00','20120229','23')
#     plus_list(a,'20120301','00','20120331','23')
#     plus_list(a,'20120401','00','20120430','23')
#     plus_list(a,'20120501','00','20120531','23')
#     plus_list(a,'20120601','00','20120630','23')
#     plus_list(a,'20120701','00','20120731','23')
#     plus_list(a,'20120801','00','20120831','23')
#     plus_list(a,'20120901','00','20120930','23')
#     plus_list(a,'20121001','00','20121031','23')
#     plus_list(a,'20121101','00','20121130','23')
#     plus_list(a,'20121201','00','20121231','23')

#     # 2013년
#     plus_list(a,'20130101','00','20130131','23')
#     plus_list(a,'20130201','00','20130228','23')
#     plus_list(a,'20130301','00','20130331','23')
#     plus_list(a,'20130401','00','20130430','23')
#     plus_list(a,'20130501','00','20130531','23')
#     plus_list(a,'20130601','00','20130630','23')
#     plus_list(a,'20130701','00','20130731','23')
#     plus_list(a,'20130801','00','20130831','23')
#     plus_list(a,'20130901','00','20130930','23')
#     plus_list(a,'20131001','00','20131031','23')
#     plus_list(a,'20131101','00','20131130','23')
#     plus_list(a,'20131201','00','20131231','23')

#     # 2014년
#     plus_list(a,'20140101','00','20140131','23')
#     plus_list(a,'20140201','00','20140228','23')
#     plus_list(a,'20140301','00','20140331','23')
#     plus_list(a,'20140401','00','20140430','23')
#     plus_list(a,'20140501','00','20140531','23')
#     plus_list(a,'20140601','00','20140619','23')
#   except:
#     print('error!')
#     print(a)

# climate2_df = pd.DataFrame(climate_datas)

# # 컬럼명
# climate2_df.columns = ['일시', '지점명', '지점번호', '기온', '강수량', '풍속', '풍향', '습도']
# climate2_df

오류났던 것들 다시 크롤링하니 결과가 잘 나옴. 거의다 21624개에 근접하게 나왔음.

In [30]:
# climate2_df.value_counts('지점명')

### 데이터 병합
- climate_df 에서 오류났던것들 삭제하고 climate2_df 데이터와 합치기

In [31]:
# climate_df = pd.read_csv('/content/drive/MyDrive/DE  DS 합반 (산불위험 프젝)/데이터셋/기상데이터_1차_초안.csv')

In [32]:
# climate_df.info()

In [33]:
# climate2_df.info()

In [34]:
# climate_df.columns

In [35]:
# 데이터 타입 통일시키기
# climate_df.loc[:,['지점번호', '기온', '강수량', '풍속', '풍향', '습도']] = climate_df.loc[:,['지점번호', '기온', '강수량', '풍속', '풍향', '습도']].astype(str)

In [36]:
# climate_df.info()

In [37]:
# climate2_df[climate2_df['지점번호'] == '95']

In [38]:
# # 데이터 삭제 (1차 데이터에서)
# a = list(climate_df[climate_df['지점번호'] == '95'].index)
# b = list(climate_df[climate_df['지점번호'] == '217'].index)
# c = list(climate_df[climate_df['지점번호'] == '273'].index)
# d = list(climate_df[climate_df['지점번호'] == '279'].index)
# e = list(climate_df[climate_df['지점번호'] == '283'].index)
# f = list(climate_df[climate_df['지점번호'] == '294'].index)

# lst = a+b+c+d+e+f

# climate_df = climate_df.drop(lst,axis=0)

In [39]:
# climate_df[climate_df['기온'] == 'nan']

In [40]:
# # 1차데이터의 nan값들이 문자열 nan으로 들어가있음. 이거 처리하기
# a_lst = []
# b_lst = []
# c_lst = []
# d_lst = []
# e_lst = []

# for a,b,c,d,e in zip(climate_df['기온'],climate_df['강수량'],climate_df['풍속'],climate_df['풍향'],climate_df['습도']):
#   if (a == 'nan') | (b == 'nan') | (c == 'nan') | (d == 'nan') | (e == 'nan'):
#     a = a.replace('nan','')
#     b = b.replace('nan','')
#     c = c.replace('nan','')
#     d = d.replace('nan','')
#     e = e.replace('nan','')

#     a_lst.append(a)
#     b_lst.append(b)
#     c_lst.append(c)
#     d_lst.append(d)
#     e_lst.append(e)
#   else:
#     a_lst.append(a)
#     b_lst.append(b)
#     c_lst.append(c)
#     d_lst.append(d)
#     e_lst.append(e)

# climate_df['기온'] = a_lst
# climate_df['강수량'] = b_lst
# climate_df['풍속'] = c_lst
# climate_df['풍향'] = d_lst
# climate_df['습도'] = e_lst

In [41]:
# climate_df[climate_df['강수량'] == 'nan']

In [42]:
# a = climate_df.value_counts('지점명')
# b = pd.DataFrame(a, columns=['갯수'])
# b[b['갯수'] != 21624]
# # b[b['갯수'] < 20000]

이제 데이터 형식도 완전 같아 졌으니 데이터 병합!

In [43]:
# climate_concat_df = pd.concat([climate_df, climate2_df], axis=0)
# climate_concat_df

In [44]:
# # 정렬하기위해 지점번호를 int형으로 변환해야됨.
# climate_concat_df['지점번호'] = climate_concat_df['지점번호'].astype(int)
# climate_concat_df = climate_concat_df.sort_values(by = ['지점번호','일시']).reset_index(drop=True)
# climate_concat_df

In [45]:
# # 저장
# climate_concat_df.to_csv('기상데이터_2차(0707).csv', index=False)
# files.download('기상데이터_2차(0707).csv')

In [46]:
# climate_concat_df.info()

## 실효습도 크롤링

In [47]:
he_df = pd.read_csv('/content/drive/MyDrive/DE  DS 합반 (산불위험 프젝)/데이터셋/실효습도_크롤링_2012.01-2014.06.csv')
he_df

,지점,지점명,일시,실효습도
0,90,속초,2012-01-01 00:00,47.44
1,90,속초,2012-01-01 01:00,45.95
2,90,속초,2012-01-01 02:00,44.15
3,90,속초,2012-01-01 03:00,43.85
4,90,속초,2012-01-01 04:00,45.00
...,...,...,...,...
1993904,295,남해,2014-06-19 19:00,54.27
1993905,295,남해,2014-06-19 20:00,61.35
1993906,295,남해,2014-06-19 21:00,64.00
1993907,295,남해,2014-06-19 22:00,64.21


In [48]:
he_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1993909 entries, 0 to 1993908
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   지점      int64  
 1   지점명     object 
 2   일시      object 
 3   실효습도    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 60.8+ MB


In [49]:
# ASOS 관측지점이 아닌곳들의 데이터는 삭제
# 116(관악산), 176(대구(기)), 214(삼척), 164(무안), 256(주암), 175(진도(첨찰산))
# 187(성산), 265(성산포)
# 268(진도군) -> 얘는 ASOS에 존재하긴하는데 2014.05.09 부터 관측을 시작해서 데이터가 한달치 정도밖에 없음. 그냥 삭제

a = list(he_df[he_df['지점'] == 176].index) # 176
b = list(he_df[he_df['지점'] == 256].index) # 256
c = list(he_df[he_df['지점'] == 175].index) # 175
d = list(he_df[he_df['지점'] == 268].index) # 268

lst = a+b+c+d

he_df = he_df.drop(lst, axis=0)
he_df

,지점,지점명,일시,실효습도
0,90,속초,2012-01-01 00:00,47.44
1,90,속초,2012-01-01 01:00,45.95
2,90,속초,2012-01-01 02:00,44.15
3,90,속초,2012-01-01 03:00,43.85
4,90,속초,2012-01-01 04:00,45.00
...,...,...,...,...
1993904,295,남해,2014-06-19 19:00,54.27
1993905,295,남해,2014-06-19 20:00,61.35
1993906,295,남해,2014-06-19 21:00,64.00
1993907,295,남해,2014-06-19 22:00,64.21


In [50]:
# 병합하기 위해 컬럼명 변경
he_df.rename(columns={'지점' : '지점번호'}, inplace=True)

In [51]:
climate_final_df = pd.read_csv('/content/drive/MyDrive/DE  DS 합반 (산불위험 프젝)/데이터셋/기상데이터_2차(0707).csv')
climate_final_df

,일시,지점명,지점번호,기온,강수량,풍속,풍향,습도
0,2012-01-01 00:00,속초,90,-0.5,NaN,0.9,50.0,57.0
1,2012-01-01 01:00,속초,90,0.3,NaN,1.2,290.0,54.0
2,2012-01-01 02:00,속초,90,0.3,NaN,1.4,320.0,53.0
3,2012-01-01 03:00,속초,90,0.5,NaN,1.5,290.0,52.0
4,2012-01-01 04:00,속초,90,0.3,NaN,1.5,290.0,51.0
...,...,...,...,...,...,...,...,...
1967383,2014-06-19 19:00,남해,295,24.5,NaN,0.9,270.0,67.0
1967384,2014-06-19 20:00,남해,295,22.3,NaN,0.9,250.0,79.0
1967385,2014-06-19 21:00,남해,295,21.5,NaN,0.7,250.0,78.0
1967386,2014-06-19 22:00,남해,295,21.5,NaN,1.2,250.0,79.0


In [52]:
climate_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1967388 entries, 0 to 1967387
Data columns (total 8 columns):
 #   Column  Dtype  
---  ------  -----  
 0   일시      object 
 1   지점명     object 
 2   지점번호    int64  
 3   기온      float64
 4   강수량     float64
 5   풍속      float64
 6   풍향      float64
 7   습도      float64
dtypes: float64(5), int64(1), object(2)
memory usage: 120.1+ MB


In [53]:
len(he_df['지점번호'].unique()) == len(climate_final_df['지점번호'].unique())

True

### 데이터 병합

In [54]:
for a in list(he_df['지점번호'].unique()):
  globals()[f'test1_{a}'] = climate_final_df[climate_final_df['지점번호'] == a]
  globals()[f'test2_{a}'] = he_df[he_df['지점번호'] == a].drop(['지점번호', '지점명'], axis=1)

  globals()[f'test_{a}_df'] = pd.merge(globals()[f'test1_{a}'], globals()[f'test2_{a}'], on = '일시', how = 'outer')
  globals()[f'test_{a}_df'] = globals()[f'test_{a}_df'].drop_duplicates('일시', keep='first')  # 중복데이터 삭제

test_all_df = pd.concat([test_90_df,test_95_df], axis=0)

for b in list(he_df['지점번호'].unique())[2:]:
  test_all_df = pd.concat([test_all_df,globals()[f'test_{b}_df']], axis=0)

데이터가 이상하게 뻥튀기되서 나왔길래 확인해보니 실효습도를 수집할때 인터넷 연결이 잠깐 몇초 끊겼나보다. 데이터가 중복으로 크롤링된 흔적을 확인했다. 
- ex) 2012-03-23일이 여러번 존재 03-24, 03-25 일은 없음.
        
        - 뻥튀기되어 나온 test_all_df 갯수 : 2015271
        - 원래 나왔어야할 데이터갯수 : 1967388
        - 일단 중복제거하니 1967388로 잘 나오긴 했는데 그럼 몇몇 실효습도는 NaN으로 들어갔을것임.
        - NaN 값 나온 날짜들 실효습도 다시 크롤링


In [55]:
test_all_df

,일시,지점명,지점번호,기온,강수량,풍속,풍향,습도,실효습도
0,2012-01-01 00:00,속초,90,-0.5,NaN,0.9,50.0,57.0,47.44
1,2012-01-01 01:00,속초,90,0.3,NaN,1.2,290.0,54.0,45.95
2,2012-01-01 02:00,속초,90,0.3,NaN,1.4,320.0,53.0,44.15
3,2012-01-01 03:00,속초,90,0.5,NaN,1.5,290.0,52.0,43.85
4,2012-01-01 04:00,속초,90,0.3,NaN,1.5,290.0,51.0,45.00
...,...,...,...,...,...,...,...,...,...
22147,2014-06-19 19:00,남해,295,24.5,NaN,0.9,270.0,67.0,54.27
22148,2014-06-19 20:00,남해,295,22.3,NaN,0.9,250.0,79.0,61.35
22149,2014-06-19 21:00,남해,295,21.5,NaN,0.7,250.0,78.0,64.00
22150,2014-06-19 22:00,남해,295,21.5,NaN,1.2,250.0,79.0,64.21


In [56]:
# NaN 값들이 들어간 날짜 추출
# 일단 속초(90)으로 확인
# 차피 크롤링이 중복된거면 속초에서 NaN값인 실효습도는 다른지점에서도 NaN값 일것임.

a_lst = []

for b in list(he_df['지점번호'].unique()):

  test_lst = list(globals()[f'test_{b}_df'].loc[globals()[f'test_{b}_df']['실효습도'].isnull() == True, '일시'])

  for a in test_lst:
    # a = a[:-6]
    a_lst.append(a)

set(a_lst)
# len(set(a_lst))

# # txt로 다운 받아서 숫자가 연속적이지 않은 날짜 데이터는 중복에 의한 NaN값이 아니라 no data에 의한 NaN값으로 판정
# b=[]
# b.append(list(set(a_lst)))

# pd.DataFrame(b).T.sort_values(by=0).to_csv('실효습도_NaN_리스트.txt', index=False)
# files.download('실효습도_NaN_리스트.txt')

{'2014-01-26 15:00',
 '2013-08-30 01:00',
 '2014-01-14 08:00',
 '2014-02-04 02:00',
 '2014-02-03 11:00',
 '2012-11-16 05:00',
 '2014-03-30 12:00',
 '2012-10-09 14:00',
 '2012-07-03 21:00',
 '2012-03-25 07:00',
 '2013-11-18 21:00',
 '2013-06-16 00:00',
 '2012-08-15 05:00',
 '2013-06-01 01:00',
 '2013-10-05 03:00',
 '2013-10-03 20:00',
 '2014-01-14 10:00',
 '2012-11-16 01:00',
 '2012-04-23 05:00',
 '2012-04-12 16:00',
 '2013-01-25 14:00',
 '2012-08-30 06:00',
 '2014-01-30 22:00',
 '2012-04-21 10:00',
 '2013-06-16 18:00',
 '2013-06-15 17:00',
 '2012-08-12 08:00',
 '2012-04-11 11:00',
 '2012-03-21 01:00',
 '2012-03-31 05:00',
 '2013-03-19 17:00',
 '2012-07-01 09:00',
 '2014-01-15 15:00',
 '2013-08-08 03:00',
 '2014-05-01 04:00',
 '2014-01-25 16:00',
 '2013-09-30 03:00',
 '2014-01-29 16:00',
 '2013-08-28 10:00',
 '2012-11-09 08:00',
 '2013-07-31 20:00',
 '2012-07-25 17:00',
 '2012-07-01 14:00',
 '2013-08-30 16:00',
 '2014-05-01 23:00',
 '2013-08-31 16:00',
 '2013-01-25 16:00',
 '2014-01-13 

In [57]:
len(test_all_df['지점번호'].unique())

91

In [58]:
# 행 갯수가 91개 나오는 날짜 찾기 -> 전 지점이 모두 NaN 이라는건 크롤링 자체를 못해왔다는 의미니까
test_all_df[(test_all_df['실효습도'].isnull() == True ) & (test_all_df['일시'] == '2012-04-01 01:00')]

,일시,지점명,지점번호,기온,강수량,풍속,풍향,습도,실효습도
2329,2012-04-01 01:00,속초,90,3.7,NaN,1.8,270.0,38.0,NaN
2329,2012-04-01 01:00,철원,95,0.8,NaN,3.0,250.0,37.0,NaN
2329,2012-04-01 01:00,동두천,98,1.1,NaN,4.0,290.0,40.0,NaN
2317,2012-04-01 01:00,파주,99,0.9,NaN,0.6,360.0,46.0,NaN
2329,2012-04-01 01:00,대관령,100,-2.9,NaN,4.0,230.0,50.0,NaN
...,...,...,...,...,...,...,...,...,...
2329,2012-04-01 01:00,합천,285,4.2,NaN,3.6,340.0,31.0,NaN
2329,2012-04-01 01:00,밀양,288,5.3,NaN,4.2,340.0,42.0,NaN
2329,2012-04-01 01:00,산청,289,2.8,NaN,4.9,290.0,38.0,NaN
2329,2012-04-01 01:00,거제,294,5.5,NaN,3.5,290.0,33.0,NaN


### 실효습도 크롤링 버그 해결
- 2012-03-20, 2012-03-21 ✔
- 2012-03-26, 2012-03-27 ✔
- 2012-04-01, 2012-04-02 ✔
- 2012-09-25 ✔
- 2012-11-07 ✔
- 2012-11-09 ✔
- 2012-11-16 ✔

- 2013-01-24, 2013-01-25 ✔
- 2013-02-08 ✔
- 2013-02-12 ✔
- 2013-02-17 ✔
- 2013-02-28 ✔
- 2013-04-18 ✔
- 2013-06-15, 2013-06-16 ✔

- 2014-03-29, 2014-03-30 ✔
- 2014-04-17 ✔



In [59]:
nan_he_df = pd.read_csv('/content/drive/MyDrive/DE  DS 합반 (산불위험 프젝)/데이터셋/실효습도_2차_NaN_크롤링.csv')
nan_he_df

,지점,지점명,일시,실효습도
0,90,속초,2012-03-20 00:00,58.52
1,90,속초,2012-03-20 01:00,59.12
2,90,속초,2012-03-20 02:00,58.93
3,90,속초,2012-03-20 03:00,59.81
4,90,속초,2012-03-20 04:00,60.23
...,...,...,...,...
48560,295,남해,2014-04-17 19:00,49.03
48561,295,남해,2014-04-17 20:00,51.90
48562,295,남해,2014-04-17 21:00,52.86
48563,295,남해,2014-04-17 22:00,57.31


he_df 다시 불러와서 nan_he_df와 합쳐서 다시 climate_df이랑 합치는게 제일 편할듯함.

In [60]:
he_df = pd.read_csv('/content/drive/MyDrive/DE  DS 합반 (산불위험 프젝)/데이터셋/실효습도_크롤링_2012.01-2014.06.csv')

he_df = pd.concat([he_df, nan_he_df], axis=0)

# 컬럼명 통일
he_df.rename(columns={'지점' : '지점번호'}, inplace=True)

# ASOS 관측지점이 아닌곳들의 데이터는 삭제
# 116(관악산), 176(대구(기)), 214(삼척), 164(무안), 256(주암), 175(진도(첨찰산))
# 187(성산), 265(성산포)
# 268(진도군) -> 얘는 ASOS에 존재하긴하는데 2014.05.09 부터 관측을 시작해서 데이터가 한달치 정도밖에 없음. 그냥 삭제

a = list(he_df[he_df['지점번호'] == 176].index) # 176
b = list(he_df[he_df['지점번호'] == 256].index) # 256
c = list(he_df[he_df['지점번호'] == 175].index) # 175
d = list(he_df[he_df['지점번호'] == 268].index) # 268

lst = a+b+c+d

he_df = he_df.drop(lst, axis=0)

# 중복 데이터 제거하고 정렬시키기
he_df.drop_duplicates(inplace=True)
he_df.sort_values(by=['지점번호','일시'], inplace=True)

# 데이터 병합
# 데이터 변형됬을수있으니 다시 불러오기
climate_final_df = pd.read_csv('/content/drive/MyDrive/DE  DS 합반 (산불위험 프젝)/데이터셋/기상데이터_2차(0707).csv')

for a in list(he_df['지점번호'].unique()):
  globals()[f'test1_{a}'] = climate_final_df[climate_final_df['지점번호'] == a]
  globals()[f'test2_{a}'] = he_df[he_df['지점번호'] == a].drop(['지점번호', '지점명'], axis=1)

  globals()[f'test_{a}_df'] = pd.merge(globals()[f'test1_{a}'], globals()[f'test2_{a}'], on = '일시', how = 'outer')

test_all_df = pd.concat([test_90_df,test_95_df], axis=0)

for b in list(he_df['지점번호'].unique())[2:]:
  test_all_df = pd.concat([test_all_df,globals()[f'test_{b}_df']], axis=0)

# 인덱스 재정렬
test_all_df.reset_index(drop=True, inplace=True)
he_df.reset_index(drop=True, inplace=True)

In [61]:
test_all_df

,일시,지점명,지점번호,기온,강수량,풍속,풍향,습도,실효습도
0,2012-01-01 00:00,속초,90,-0.5,NaN,0.9,50.0,57.0,47.44
1,2012-01-01 01:00,속초,90,0.3,NaN,1.2,290.0,54.0,45.95
2,2012-01-01 02:00,속초,90,0.3,NaN,1.4,320.0,53.0,44.15
3,2012-01-01 03:00,속초,90,0.5,NaN,1.5,290.0,52.0,43.85
4,2012-01-01 04:00,속초,90,0.3,NaN,1.5,290.0,51.0,45.00
...,...,...,...,...,...,...,...,...,...
1967383,2014-06-19 19:00,남해,295,24.5,NaN,0.9,270.0,67.0,54.27
1967384,2014-06-19 20:00,남해,295,22.3,NaN,0.9,250.0,79.0,61.35
1967385,2014-06-19 21:00,남해,295,21.5,NaN,0.7,250.0,78.0,64.00
1967386,2014-06-19 22:00,남해,295,21.5,NaN,1.2,250.0,79.0,64.21


In [62]:
# NaN 값 없는지 테스트
# 원래는 91개의 행이 나왔던 코드
test_all_df[(test_all_df['실효습도'].isnull() == True ) & (test_all_df['일시'] == '2012-04-01 01:00')]

,일시,지점명,지점번호,기온,강수량,풍속,풍향,습도,실효습도
931996,2012-04-01 01:00,진주,192,3.4,NaN,3.5,360.0,35.0,NaN
1429213,2012-04-01 01:00,북창원,255,4.7,NaN,3.7,320.0,NaN,NaN
1558911,2012-04-01 01:00,고흥,262,2.6,NaN,3.5,320.0,45.0,NaN


In [63]:
# # 저장
# he_df.to_csv('실효습도_final.csv', index=False)
# files.download('실효습도_final.csv')

## 라벨링

In [64]:
fire_df = pd.read_csv('/content/drive/MyDrive/DE  DS 합반 (산불위험 프젝)/데이터셋/mongo_db,화재사건,데이터(0701.ver).csv')
fire_df

,_id,발생일시,발생일시(요일),진화일시(년월일시분),진화소요시간(분),발생장소(관서),발생장소(시도),발생장소(시군구),발생장소(읍면동),발생장소(리),...,평균기온,실효습도,상대습도,최소습도,최대풍속,평균풍속,최대풍향,평균풍향,강우결과일수,최종강우량
0,62be5e0852482b6dab131e1d,201406191500,목,201406191800,300,북부,강원,인제,남,남전,...,23.6,0.0,57.0,0.0,5.6,1.9,S,SE,6,0.5
1,62be5e0852482b6dab131e1e,201406181805,수,201406182015,210,경남,경남,밀양,삼랑진,미전,...,29.6,0.0,46.0,0.0,4.0,2.4,SE,NNE,6,1.5
2,62be5e0852482b6dab131e1f,201406151410,일,201406151720,310,강원,강원,인제,인제,가아,...,29.3,0.0,57.0,0.0,6.7,1.3,SSW,NW,2,0.5
3,62be5e0852482b6dab131e20,201406141720,토,201406141900,140,전북,전북,부안,보안,신복,...,25.3,0.0,46.0,0.0,7.8,3.3,W,WNW,3,1.5
4,62be5e0852482b6dab131e21,201406101718,화,201406102000,242,북부,강원,원주,흥업,매지,...,22.3,0.0,40.0,0.0,9.9,2.0,SSE,NNE,1,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11719,62be5e0852482b6dab134be4,191107022013,,,,대전,대전,중,대사,,...,0.0,0.0,0.0,0.0,0.0,0.0,,,0,0.0
11720,62be5e0852482b6dab134be5,191107022013,월,191101010300,1225,대전,대전,중,대사,,...,0.0,0.0,0.0,0.0,0.0,0.0,,,0,0.0
11721,62be5e0852482b6dab134be6,191106290400,,,,대전,대전,동,천,,...,0.0,0.0,0.0,0.0,0.0,0.0,,,0,0.0
11722,62be5e0852482b6dab134be7,191106290400,월,191101010000,1225,대전,대전,동,천,,...,0.0,0.0,0.0,0.0,0.0,0.0,,,0,0.0


In [65]:
fire_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11724 entries, 0 to 11723
Data columns (total 27 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   _id          11724 non-null  object 
 1   발생일시         11724 non-null  int64  
 2   발생일시(요일)     11724 non-null  object 
 3   진화일시(년월일시분)  11724 non-null  object 
 4   진화소요시간(분)    11724 non-null  object 
 5   발생장소(관서)     11724 non-null  object 
 6   발생장소(시도)     11724 non-null  object 
 7   발생장소(시군구)    11724 non-null  object 
 8   발생장소(읍면동)    11724 non-null  object 
 9   발생장소(리)      11724 non-null  object 
 10  발생장소(지번)     11724 non-null  object 
 11  소유구분         11724 non-null  object 
 12  발생세부원인       11724 non-null  object 
 13  피해면적         11724 non-null  float64
 14  피해액          11724 non-null  int64  
 15  산불위험지수(평균)   11724 non-null  float64
 16  산불위험지수(최대)   11724 non-null  float64
 17  평균기온         11724 non-null  float64
 18  실효습도         11724 non-null  float64
 19  상대습도

In [66]:
# 2012 ~ 2014년 사건만 추리기
event3_df = fire_df[fire_df['발생일시'] > 201200000000]
event3_df = event3_df.drop('_id', axis=1).sort_values(by='발생일시').reset_index(drop=True)
event3_df

,발생일시,발생일시(요일),진화일시(년월일시분),진화소요시간(분),발생장소(관서),발생장소(시도),발생장소(시군구),발생장소(읍면동),발생장소(리),발생장소(지번),...,평균기온,실효습도,상대습도,최소습도,최대풍속,평균풍속,최대풍향,평균풍향,강우결과일수,최종강우량
0,201201051139,목,201201051304,125,부산,부산,동,초량,,산25-1,...,0.2,35.0,27.0,0.0,8.4,6.0,NNW,N,33,3.0
1,201201091215,월,201201091340,125,경북,경북,영덕,지품,오천,산4,...,4.2,32.4,38.0,0.0,6.0,4.6,NW,WNW,10,1.5
2,201201111430,수,201201111615,145,경기,경기,남양주,호평,,산5-29,...,-3.9,45.5,30.0,0.0,7.6,3.2,WSW,NW,18,0.5
3,201201111515,수,201201111650,135,북부,강원,홍천,홍천,하오안,산30-2,...,-0.9,50.8,22.0,0.0,0.9,1.3,WNW,WNW,18,0.5
4,201201142215,토,201201150940,1125,전남,전남,완도,금일,동백,산258,...,0.9,42.3,33.0,0.0,7.8,0.7,NNW,W,10,2.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896,201406101718,화,201406102000,242,북부,강원,원주,흥업,매지,산66,...,22.3,0.0,40.0,0.0,9.9,2.0,SSE,NNE,1,0.5
897,201406141720,토,201406141900,140,전북,전북,부안,보안,신복,산65,...,25.3,0.0,46.0,0.0,7.8,3.3,W,WNW,3,1.5
898,201406151410,일,201406151720,310,강원,강원,인제,인제,가아,산70,...,29.3,0.0,57.0,0.0,6.7,1.3,SSW,NW,2,0.5
899,201406181805,수,201406182015,210,경남,경남,밀양,삼랑진,미전,산123-1,...,29.6,0.0,46.0,0.0,4.0,2.4,SE,NNE,6,1.5


### 카카오 API를 이용해 산불사건 위경도 구하기
1. 산불사건 발생지역의 위경도를 구한다.
2. 기상관측지점의 위경도 파일셋을 불러온다.
3. 산불사건 발생지역에서 제일 가까운 기상관측지점을 찾는다.
  - 유클리디안 거리 사용
  - from scipy.spatial import distance 사용

```python
event3_df['주소'] = event3_df['발생장소(시도)'] + ' ' +  event3_df['발생장소(시군구)'] + ' ' + event3_df['발생장소(리)'] + '리 ' + event3_df['발생장소(지번)']
```
위 코드로 실행해서 주소 for문 돌리니까  

901개의 데이터 중 259개의 데이터가 주소 오류가 떳다. 코드 자체를 바꿔야할듯. 아니면 주소를 다른방식으로 변형하든가

In [67]:
# 산불사건 발생지역의 주소를 구하기
event3_df['주소'] = event3_df['발생장소(시도)'] + ' ' +  event3_df['발생장소(시군구)'] + ' ' + event3_df['발생장소(읍면동)']
event3_df['주소']

0        부산 동 초량
1       경북 영덕 지품
2      경기 남양주 호평
3       강원 홍천 홍천
4       전남 완도 금일
         ...    
896     강원 원주 흥업
897     전북 부안 보안
898     강원 인제 인제
899    경남 밀양 삼랑진
900      강원 인제 남
Name: 주소, Length: 901, dtype: object

In [68]:
# 카카오 API 세팅

KAKAO_API_KEY = "3644bda5f1ec337a30bc034d4e383a0a"
KAKAO_LOCAL_URL = "https://dapi.kakao.com/v2/local/search/address?query={}"

header = {
"Authorization": "KakaoAK {}".format(KAKAO_API_KEY)
}

longitude  = []
latitude = []
error_lst=[]
i = 0

for query in list(event3_df['주소']):

  try:
    response = requests.get(KAKAO_LOCAL_URL.format(query), headers=header)
    datas = response.json()

    # 데이터가 제대로 나왔는지 검사 -> 부산에 있는 '부산 동'을 검색하면 '경기 오산시 부산동' 이 나옴. 그러니까 결과가 제대로 나왔는지 검사해야됨.
    re.search(query.split()[0], datas['documents'][0]['address']['address_name'].split()[0]).group()  # 검색결과가 잘못나왔다면 에러 발생

    longitude.append(datas['documents'][0]['address']['x'])  # 경도
    latitude.append(datas['documents'][0]['address']['y'] ) # 위도

  except:
    try:
      # '부산 동 초량' 이 형식의 주소에서 검색했을시 결과가 안나온다면 '부산 동' 까지만 검색해보도록 하기
      query = query.split()[0] +' ' + query.split()[1] 
      response = requests.get(KAKAO_LOCAL_URL.format(query), headers=header)
      datas = response.json()

      # 주소 검색결과 유효성 검사
      re.search(query.split()[0], datas['documents'][0]['address']['address_name'].split()[0]).group()

      longitude.append(datas['documents'][0]['address']['x'])  # 경도
      latitude.append(datas['documents'][0]['address']['y'] ) # 위도
    except:
      try:
        # 그래도 검색이 안된다면 '부산' 까지만 검색
        query = query.split()[0]
        response = requests.get(KAKAO_LOCAL_URL.format(query), headers=header)
        datas = response.json()
        
        longitude.append(datas['documents'][0]['address']['x'])  # 경도
        latitude.append(datas['documents'][0]['address']['y'] ) # 위도
      except:
        # 에러 뜬 주소들은 직접 제일 가까운 위경도 찾아서 입력해줘야 하기 때문에
        # 주소들 리스트에 담아두기
        print(query,i)
        error_lst.append(query + ' ' + f'{i}')
        longitude.append('0')
        latitude.append('0')

  i += 1

event3_df['longitude'] = longitude
event3_df['latitude'] = latitude

출북 8


카카오 API 코드 실행완료까지 12분 정도 걸림

In [69]:
# 8번 인덱스만 오류가 났음. 주소가 '충북' 인데 '출북' 으로 되어있음. 산림청에서 오타낸듯
# '충북 음성' 의 위, 경도
response = requests.get(KAKAO_LOCAL_URL.format('충북 음성'), headers=header)
datas = response.json()
print(datas['documents'][0]['address']['x'], datas['documents'][0]['address']['y'])

127.690480041094 36.9403218849121


In [70]:
# 8번 인덱스의 위경도 변경해주기
event3_df.loc[8,'longitude'] = '127.690480041094'
event3_df.loc[8,'latitude'] = '36.9403218849121'
event3_df.loc[8,['longitude','latitude']]

longitude    127.690480041094
latitude     36.9403218849121
Name: 8, dtype: object

In [71]:
event3_df.columns

Index(['발생일시', '발생일시(요일)', '진화일시(년월일시분)', '진화소요시간(분)', '발생장소(관서)', '발생장소(시도)',
       '발생장소(시군구)', '발생장소(읍면동)', '발생장소(리)', '발생장소(지번)', '소유구분', '발생세부원인',
       '피해면적', '피해액', '산불위험지수(평균)', '산불위험지수(최대)', '평균기온', '실효습도', '상대습도',
       '최소습도', '최대풍속', '평균풍속', '최대풍향', '평균풍향', '강우결과일수', '최종강우량', '주소',
       'longitude', 'latitude'],
      dtype='object')

In [122]:
# 필요한 컬럼만 사용
event_final_df = event3_df.loc[:,['발생일시','주소', 'longitude', 'latitude']]
event_final_df

,발생일시,주소,longitude,latitude
0,201201051139,부산 동 초량,129.075087492149,35.1798200522868
1,201201091215,경북 영덕 지품,129.28178296249,36.4484953265767
2,201201111430,경기 남양주 호평,127.248669169781,37.6548880029094
3,201201111515,강원 홍천 홍천,127.888684130433,37.6971673162731
4,201201142215,전남 완도 금일,127.029520006322,34.3488662894218
...,...,...,...,...
896,201406101718,강원 원주 흥업,127.921181194086,37.3025500299789
897,201406141720,전북 부안 보안,126.733011321782,35.7316577924649
898,201406151410,강원 인제 인제,128.170351448366,38.0697061389166
899,201406181805,경남 밀양 삼랑진,128.837736329474,35.395826706282


### 가까운 관측지점 찾기

In [73]:
len(test_all_df['지점번호'].unique())

91

In [123]:
# 위경도 데이터프레임이 변형되어있으니 다시 호출
latitude_df = pd.read_csv('/content/drive/MyDrive/DE  DS 합반 (산불위험 프젝)/데이터셋/mongo_db,관측지점,위경도(0701.ver).csv')

# 관측시작(설립일)이 2012~2014년 이후인 93, 177, 239, 268은 삭제하여 기상데이터(test_all_df)와 지점갯수 맞추기
latitude_df = latitude_df[(latitude_df['지점번호'] != 93) & (latitude_df['지점번호'] != 177) & (latitude_df['지점번호'] != 239) & (latitude_df['지점번호'] != 268)]
latitude_df

,_id,지점번호,지점명,위도,경도
0,62be6fb4a82199005252108d,90,속초,38.25085,128.56473
2,62be6fb4a82199005252108f,95,철원,38.14787,127.30420
3,62be6fb4a821990052521090,98,동두천,37.90188,127.06070
4,62be6fb4a821990052521091,99,파주,37.88589,126.76648
5,62be6fb4a821990052521092,100,대관령,37.67713,128.71834
...,...,...,...,...,...
90,62be6fb4a8219900525210e7,285,합천,35.56505,128.16994
91,62be6fb4a8219900525210e8,288,밀양,35.49147,128.74413
92,62be6fb4a8219900525210e9,289,산청,35.41300,127.87910
93,62be6fb4a8219900525210ea,294,거제,34.88818,128.60458


In [75]:
latitude_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91 entries, 0 to 94
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   _id     91 non-null     object 
 1   지점번호    91 non-null     int64  
 2   지점명     91 non-null     object 
 3   위도      91 non-null     float64
 4   경도      91 non-null     float64
dtypes: float64(2), int64(1), object(2)
memory usage: 4.3+ KB


In [76]:
event_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 901 entries, 0 to 900
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   발생일시       901 non-null    int64 
 1   주소         901 non-null    object
 2   longitude  901 non-null    object
 3   latitude   901 non-null    object
dtypes: int64(1), object(3)
memory usage: 28.3+ KB


In [124]:
# 위, 경도 숫자 데이터로 바꾸기
event_final_df['longitude'] = event_final_df['longitude'].astype(float)
event_final_df['latitude'] = event_final_df['latitude'].astype(float)

In [78]:
# test
tuple(latitude_df.iloc[1,3:5])

(38.14787, 127.3042)

In [125]:
from scipy.spatial import distance

min_dist = []

for c,d in zip(event_final_df['latitude'],event_final_df['longitude']):

  # for문 돌때마다 dist의 리스트가 초기화 되게끔 for문 안쪽에 배치
  dist = []

  for i in range(len(latitude_df['지점번호'])):
    globals()[f'a_{i}'] = tuple(latitude_df.iloc[i,3:5])

    b = (c,d)

    # 각 지점마다의 유클리디안 거리 구한후 리스트에 추가
    dist.append(distance.euclidean(globals()[f'a_{i}'] , b))

  # str.contains를 사용하기위해 시리즈 형태로 만들어주기
  ss = pd.Series(dist, index = latitude_df['지점명'].to_list())

  # 문자열 포함여부 판단 & 제일 가까운 관측지점
  m = str(ss.min())  # 유클리디안 거리로 구한 관측지점과의 거리 중 제일 짧은 거리
  ss = ss.astype(str)

  min_dist.append(ss[ss.str.contains(m, case=False, regex=False) == True].index[0])
  
event_final_df['근접지점'] = min_dist
event_final_df['근접지점']

0      부산
1      영덕
2      서울
3      홍천
4      완도
       ..
896    원주
897    부안
898    인제
899    밀양
900    인제
Name: 근접지점, Length: 901, dtype: object

In [105]:
event_final_df

,발생일시,주소,longitude,latitude,근접지점
0,201201051139,부산 동 초량,129.075087,35.179820,부산
1,201201091215,경북 영덕 지품,129.281783,36.448495,영덕
2,201201111430,경기 남양주 호평,127.248669,37.654888,서울
3,201201111515,강원 홍천 홍천,127.888684,37.697167,홍천
4,201201142215,전남 완도 금일,127.029520,34.348866,완도
...,...,...,...,...,...
896,201406101718,강원 원주 흥업,127.921181,37.302550,원주
897,201406141720,전북 부안 보안,126.733011,35.731658,부안
898,201406151410,강원 인제 인제,128.170351,38.069706,인제
899,201406181805,경남 밀양 삼랑진,128.837736,35.395827,밀양


### 라벨링
- event_final_df의 발생일시 핸들링
- 관측지점에 맞게 test_all_df에 라벨링

In [82]:
event_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 901 entries, 0 to 900
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   발생일시       901 non-null    int64  
 1   주소         901 non-null    object 
 2   longitude  901 non-null    float64
 3   latitude   901 non-null    float64
 4   근접지점       901 non-null    object 
dtypes: float64(2), int64(1), object(2)
memory usage: 35.3+ KB


In [83]:
test_all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1967388 entries, 0 to 1967387
Data columns (total 9 columns):
 #   Column  Dtype  
---  ------  -----  
 0   일시      object 
 1   지점명     object 
 2   지점번호    int64  
 3   기온      float64
 4   강수량     float64
 5   풍속      float64
 6   풍향      float64
 7   습도      float64
 8   실효습도    float64
dtypes: float64(6), int64(1), object(2)
memory usage: 135.1+ MB


In [112]:
# 발생일시 확인
event_final_df[event_final_df['발생일시'] > 201201160000]

,발생일시,주소,longitude,latitude,근접지점
6,201202051528,강원 고성 거진,128.452554,38.436139,속초
7,201202161405,강원 원주 소초,128.002389,37.417963,원주
8,201202161410,출북 음성 소이,127.690480,36.940322,충주
9,201202161530,전남 화순 화순,126.986502,35.064512,광주
10,201202161630,전남 강진 칠량,126.799177,34.572664,강진군
...,...,...,...,...,...
896,201406101718,강원 원주 흥업,127.921181,37.302550,원주
897,201406141720,전북 부안 보안,126.733011,35.731658,부안
898,201406151410,강원 인제 인제,128.170351,38.069706,인제
899,201406181805,경남 밀양 삼랑진,128.837736,35.395827,밀양


분 데이터를 자르면 사건 발생일자, 사건발생시간이 같은경우도 존재함을 확인. 눈으로 보이는 데이터들은 근접지점까진 같지않으나 사건발생일자, 시간, 근접지점 까지 같은 경우도 존재할수있음.

In [126]:
# test
dd = event_final_df.copy()

dd['발생일시'] = dd['발생일시'].astype(str)

datetime_data = []

for i in dd['발생일시']:
  date = i[:4] + '-' + i[4:6] + '-' + i[6:8] + ' ' + i[8:10] + ':' + i[10:]
  datetime_data.append(date)

dd['발생일시'] = datetime_data
dd['발생일시'] = pd.to_datetime(dd['발생일시'])

datetime_data = []

# 산불사건 데이터 분단위 다 버리기
for b in dd['발생일시']:
  date = b.replace(minute=00)
  
  datetime_data.append(date)

dd['발생일시'] = datetime_data

# 문자열로 파싱
dd['발생일시'] = dd['발생일시'].astype(str)

dd

,발생일시,주소,longitude,latitude,근접지점
0,2012-01-05 11:00:00,부산 동 초량,129.075087,35.179820,부산
1,2012-01-09 12:00:00,경북 영덕 지품,129.281783,36.448495,영덕
2,2012-01-11 14:00:00,경기 남양주 호평,127.248669,37.654888,서울
3,2012-01-11 15:00:00,강원 홍천 홍천,127.888684,37.697167,홍천
4,2012-01-14 22:00:00,전남 완도 금일,127.029520,34.348866,완도
...,...,...,...,...,...
896,2014-06-10 17:00:00,강원 원주 흥업,127.921181,37.302550,원주
897,2014-06-14 17:00:00,전북 부안 보안,126.733011,35.731658,부안
898,2014-06-15 14:00:00,강원 인제 인제,128.170351,38.069706,인제
899,2014-06-18 18:00:00,경남 밀양 삼랑진,128.837736,35.395827,밀양


In [127]:
# 날짜형식으로 바꾸기
event_final_df['발생일시'] = event_final_df['발생일시'].astype(str)

datetime_data = []

for i in event_final_df['발생일시']:
  date = i[:4] + '-' + i[4:6] + '-' + i[6:8] + ' ' + i[8:10] + ':' + i[10:]
  datetime_data.append(date)

event_final_df['발생일시'] = datetime_data
event_final_df['발생일시'] = pd.to_datetime(event_final_df['발생일시'])

datetime_data = []

# 산불사건 데이터 분단위 다 버리기
for b in event_final_df['발생일시']:
  date = b.replace(minute=00)
  datetime_data.append(date)

event_final_df['발생일시'] = datetime_data

# 문자열로 파싱
event_final_df['발생일시'] = event_final_df['발생일시'].astype(str)

# 두 데이터프레임이 같은 형식을 띄도록 맞추기
test_all_df['일시'] = pd.to_datetime(test_all_df['일시'])
test_all_df['일시'] = test_all_df['일시'].astype(str)

# 라벨링
test_all_df['label'] = 0

for a in list(event_final_df['근접지점'].unique()):

  for b in event_final_df.loc[(event_final_df['근접지점'] == a),'발생일시']:

    test_all_df.loc[(test_all_df['지점명'] == a) & (test_all_df['일시'] == b) ,'label'] = 1

test_all_df

,일시,지점명,지점번호,기온,강수량,풍속,풍향,습도,실효습도,label
0,2012-01-01 00:00:00,속초,90,-0.5,NaN,0.9,50.0,57.0,47.44,0
1,2012-01-01 01:00:00,속초,90,0.3,NaN,1.2,290.0,54.0,45.95,0
2,2012-01-01 02:00:00,속초,90,0.3,NaN,1.4,320.0,53.0,44.15,0
3,2012-01-01 03:00:00,속초,90,0.5,NaN,1.5,290.0,52.0,43.85,0
4,2012-01-01 04:00:00,속초,90,0.3,NaN,1.5,290.0,51.0,45.00,0
...,...,...,...,...,...,...,...,...,...,...
1967383,2014-06-19 19:00:00,남해,295,24.5,NaN,0.9,270.0,67.0,54.27,0
1967384,2014-06-19 20:00:00,남해,295,22.3,NaN,0.9,250.0,79.0,61.35,0
1967385,2014-06-19 21:00:00,남해,295,21.5,NaN,0.7,250.0,78.0,64.00,0
1967386,2014-06-19 22:00:00,남해,295,21.5,NaN,1.2,250.0,79.0,64.21,0


라벨링 코드 실행시간 엄청 오래걸렸음.

In [87]:
len(list(event_final_df['근접지점'].unique()))

87

In [128]:
# 라벨링 잘 됬는지 확인 - 901개가 나와야됨.
# 890개로 나왔는데 일시랑 관측지점이 모두 같은 경우도 몇개 있어서 그런듯.
len(test_all_df[test_all_df['label'] == 1])

890

In [129]:
test_all_df[test_all_df['label'] == 1]

,일시,지점명,지점번호,기온,강수량,풍속,풍향,습도,실효습도,label
855,2012-02-05 15:00:00,속초,90,6.2,NaN,2.2,140.0,32.0,20.68,1
6902,2012-10-14 14:00:00,속초,90,19.9,NaN,3.8,360.0,54.0,44.93,1
9064,2013-01-12 16:00:00,속초,90,4.7,NaN,5.4,250.0,49.0,31.26,1
10572,2013-03-16 12:00:00,속초,90,12.4,NaN,2.9,90.0,44.0,46.63,1
11843,2013-05-08 11:00:00,속초,90,16.5,NaN,3.8,110.0,76.0,51.75,1
...,...,...,...,...,...,...,...,...,...,...
1934238,2013-02-24 18:00:00,거제,294,7.0,NaN,2.3,200.0,29.0,24.55,1
1935146,2013-04-03 14:00:00,거제,294,16.8,NaN,3.4,230.0,34.0,32.78,1
1941683,2013-12-31 23:00:00,거제,294,8.6,NaN,3.6,270.0,44.0,39.18,1
1949548,2012-06-06 16:00:00,남해,295,27.0,NaN,1.9,230.0,49.0,40.23,1


In [130]:
test_all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1967388 entries, 0 to 1967387
Data columns (total 10 columns):
 #   Column  Dtype  
---  ------  -----  
 0   일시      object 
 1   지점명     object 
 2   지점번호    int64  
 3   기온      float64
 4   강수량     float64
 5   풍속      float64
 6   풍향      float64
 7   습도      float64
 8   실효습도    float64
 9   label   int64  
dtypes: float64(6), int64(2), object(2)
memory usage: 150.1+ MB


In [134]:
# test
a = '2014-04-06 15:00:00'
a[:-3]

'2014-04-06 15:00'

In [135]:
# 데이터 정리
a_lst = []

for a in test_all_df['일시']:
  a = a[:-3]
  a_lst.append(a)

test_all_df['일시'] = a_lst
test_all_df = test_all_df.drop('지점번호' ,axis=1)

test_all_df

,일시,지점명,기온,강수량,풍속,풍향,습도,실효습도,label
0,2012-01-01 00:00,속초,-0.5,NaN,0.9,50.0,57.0,47.44,0
1,2012-01-01 01:00,속초,0.3,NaN,1.2,290.0,54.0,45.95,0
2,2012-01-01 02:00,속초,0.3,NaN,1.4,320.0,53.0,44.15,0
3,2012-01-01 03:00,속초,0.5,NaN,1.5,290.0,52.0,43.85,0
4,2012-01-01 04:00,속초,0.3,NaN,1.5,290.0,51.0,45.00,0
...,...,...,...,...,...,...,...,...,...
1967383,2014-06-19 19:00,남해,24.5,NaN,0.9,270.0,67.0,54.27,0
1967384,2014-06-19 20:00,남해,22.3,NaN,0.9,250.0,79.0,61.35,0
1967385,2014-06-19 21:00,남해,21.5,NaN,0.7,250.0,78.0,64.00,0
1967386,2014-06-19 22:00,남해,21.5,NaN,1.2,250.0,79.0,64.21,0


In [141]:
test_all_df[test_all_df['풍속'].isna() == True]

,일시,지점명,기온,강수량,풍속,풍향,습도,실효습도,label
1338,2012-02-25 18:00,속초,-0.1,1.9,NaN,NaN,99.0,70.58,0
1339,2012-02-25 19:00,속초,-0.2,NaN,NaN,NaN,99.0,71.36,0
1340,2012-02-25 20:00,속초,-0.4,NaN,NaN,NaN,99.0,71.16,0
1341,2012-02-25 21:00,속초,-0.3,2.3,NaN,NaN,99.0,73.08,0
1342,2012-02-25 22:00,속초,-0.4,NaN,NaN,NaN,100.0,72.92,0
...,...,...,...,...,...,...,...,...,...
1762958,2013-05-05 16:00,의성,25.6,NaN,NaN,NaN,18.0,22.86,0
1810891,2013-11-16 21:00,영천,NaN,NaN,NaN,NaN,NaN,NaN,0
1884892,2012-06-14 19:00,밀양,22.2,NaN,NaN,NaN,59.0,47.72,0
1884894,2012-06-14 22:00,밀양,19.9,NaN,NaN,NaN,68.0,55.97,0


In [143]:
# # 저장
# test_all_df.to_csv('2012-2014,학습데이터.csv',index=False)
# files.download('2012-2014,학습데이터.csv')

## 2012-2014,학습데이터.csv 데이터 잘 불러와지는지 확인

In [145]:
final = pd.read_csv('/content/drive/MyDrive/DE  DS 합반 (산불위험 프젝)/데이터셋/2012-2014,학습데이터.csv')
final

,일시,지점명,기온,강수량,풍속,풍향,습도,실효습도,label
0,2012-01-01 00:00,속초,-0.5,NaN,0.9,50.0,57.0,47.44,0
1,2012-01-01 01:00,속초,0.3,NaN,1.2,290.0,54.0,45.95,0
2,2012-01-01 02:00,속초,0.3,NaN,1.4,320.0,53.0,44.15,0
3,2012-01-01 03:00,속초,0.5,NaN,1.5,290.0,52.0,43.85,0
4,2012-01-01 04:00,속초,0.3,NaN,1.5,290.0,51.0,45.00,0
...,...,...,...,...,...,...,...,...,...
1967383,2014-06-19 19:00,남해,24.5,NaN,0.9,270.0,67.0,54.27,0
1967384,2014-06-19 20:00,남해,22.3,NaN,0.9,250.0,79.0,61.35,0
1967385,2014-06-19 21:00,남해,21.5,NaN,0.7,250.0,78.0,64.00,0
1967386,2014-06-19 22:00,남해,21.5,NaN,1.2,250.0,79.0,64.21,0


In [146]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1967388 entries, 0 to 1967387
Data columns (total 9 columns):
 #   Column  Dtype  
---  ------  -----  
 0   일시      object 
 1   지점명     object 
 2   기온      float64
 3   강수량     float64
 4   풍속      float64
 5   풍향      float64
 6   습도      float64
 7   실효습도    float64
 8   label   int64  
dtypes: float64(6), int64(1), object(2)
memory usage: 135.1+ MB
